# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [406]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean, pdist, squareform

In [407]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [408]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [409]:
customer = data.groupby(['CustomerID','ProductName'])[['Quantity']].sum()
customer

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [410]:
table = pd.pivot_table(customer, values='Quantity', index=['ProductName'], 
                       columns=['CustomerID'], aggfunc=np.sum, fill_value=0)
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


In [411]:
t200 = table[200]
t200 = t200[(t200 == 0)]
t200

ProductName
Anchovy Paste - 56 G Tube            0
Appetizer - Mini Egg Roll, Shrimp    0
Appetizer - Mushroom Tart            0
Appetizer - Sausage Rolls            0
Apricots - Dried                     0
                                    ..
Wonton Wrappers                      0
Yeast Dry - Fermipan                 0
Yoghurt Tubes                        0
Yogurt - French Vanilla              0
Zucchini - Yellow                    0
Name: 200, Length: 380, dtype: int64

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [412]:
#distances = pdist(table.T, 'euclidean')
#print(distances)
#print("-----Squareform-----")
#print(squareform(distances))

# Uso el tableDistance para poder obtener las distancias con valores entre 0 y 1.
tableDistances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)
tableDistances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [413]:
similares = tableDistances[200].sort_values(ascending=False)[1:6]
similares

CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [414]:
lst_cust = list(similares.index)
data_step5 = customer.loc[lst_cust]
data_step5

Quantity
CustomerID ProductName                                
1072       Appetizer - Mini Egg Roll, Shrimp         1
           Appetizer - Sausage Rolls                 1
           Beans - Kidney, Canned                    1
           Beef - Montreal Smoked Brisket            1
           Beef - Striploin Aa                       1
...                                                ...
3909       Wine - Charddonnay Errazuriz              2
           Wine - Fume Blanc Fetzer                  1
           Wine - Magnotta, Merlot Sr Vqa            1
           Wine - Toasted Head                       1
           Wine - Valpolicella Masi                  1

[262 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [415]:
step6 = data_step5.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
step6[0:5]

ProductName
Soup - Campbells Bean Medley       4
Muffin - Carrot Individual Wrap    3
Bay Leaf                           3
Pork - Kidney                      3
Pepper - Black, Whole              3
Name: Quantity, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [416]:
step7 = pd.merge(t200, step6, left_index=True, right_index=True).sort_values(by="Quantity",ascending=False)
step7 = step7.reset_index()[0:5]
step7

,ProductName,200,Quantity
0,Soup - Campbells Bean Medley,0,4
1,Muffin - Carrot Individual Wrap,0,3
2,Wanton Wrap,0,3
3,Bay Leaf,0,3
4,Pork - Kidney,0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [417]:
dictClean = {}
listaCustomers = data['CustomerID'].unique()
listaCustomers = sorted(listaCustomers)
for x in listaCustomers[:7]:
    similares = tableDistances[x].sort_values(ascending=False)[1:7]
    lst_cust = list(similares.index)
    data_step5 = customer.loc[lst_cust]
    step6 = data_step5.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
    tx = table[x]
    tx = tx[(tx==0)]
    step7 = pd.merge(tx, step6, left_index=True, right_index=True).sort_values(by="Quantity",ascending=False)
    re={x:step7.index[0:5]}
    dictClean.update(re)
    
dictClean

{33: Index(['Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein',
        'Knife Plastic - White', 'Beef Wellington',
        'Wine - Ej Gallo Sierra Valley'],
       dtype='object', name='ProductName'),
 200: Index(['Soup - Campbells Bean Medley', 'Muffin - Carrot Individual Wrap',
        'Dried Figs', 'Pork - Kidney', 'Wanton Wrap'],
       dtype='object', name='ProductName'),
 264: Index(['Veal - Inside, Choice', 'Soupfoamcont12oz 112con',
        'Pepper - Black, Whole', 'Lamb - Ground',
        'Bread - Italian Roll With Herbs'],
       dtype='object', name='ProductName'),
 356: Index(['Butter - Unsalted', 'Veal - Inside, Choice', 'Sardines',
        'Nut - Chestnuts, Whole', 'Lamb - Ground'],
       dtype='object', name='ProductName'),
 412: Index(['Wine - Blue Nun Qualitatswein', 'Soup - Campbells Bean Medley',
        'Olive - Spread Tapenade', 'Sprouts - Baby Pea Tendrils',
        'Wine - Chardonnay South'],
       dtype='object', name='ProductName'),
 464: Index(

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [418]:
newDataFrame = pd.DataFrame(dictClean)
newDataFrame

,33,200,264,356,412,464,477
0,Soup - Campbells Bean Medley,Soup - Campbells Bean Medley,"Veal - Inside, Choice",Butter - Unsalted,Wine - Blue Nun Qualitatswein,Butter - Unsalted,"Pepper - Black, Whole"
1,Wine - Blue Nun Qualitatswein,Muffin - Carrot Individual Wrap,Soupfoamcont12oz 112con,"Veal - Inside, Choice",Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Sprouts - Baby Pea Tendrils
2,Knife Plastic - White,Dried Figs,"Pepper - Black, Whole",Sardines,Olive - Spread Tapenade,Knife Plastic - White,Wine - Toasted Head
3,Beef Wellington,Pork - Kidney,Lamb - Ground,"Nut - Chestnuts, Whole",Sprouts - Baby Pea Tendrils,"Sauce - Gravy, Au Jus, Mix","Cheese - Brie, Triple Creme"
4,Wine - Ej Gallo Sierra Valley,Wanton Wrap,Bread - Italian Roll With Herbs,Lamb - Ground,Wine - Chardonnay South,Yoghurt Tubes,"Oranges - Navel, 72"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [419]:
tableDistances2 = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'correlation'))), 
                         index=table.columns, columns=table.columns)
tableDistances2

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.479764,0.484514,0.525278,0.496086,0.500064,0.507963,0.504739,0.499300,0.500268,0.492619,...,0.494577,0.504692,0.487604,0.510055,0.534474,1.000000,0.525583,0.508923,0.492050,0.482921
98069,0.493197,0.534491,0.511845,0.484207,0.481259,0.484219,0.496892,0.490306,0.495330,0.506038,...,0.489353,0.499742,0.493332,0.484621,0.518025,0.525583,1.000000,0.511707,0.501063,0.498897
98159,0.486457,0.500253,0.503500,0.502204,0.502418,0.499195,0.500240,0.502118,0.510333,0.502173,...,0.505074,0.522726,0.500553,0.489177,0.506341,0.508923,0.511707,1.000000,0.504367,0.495580


In [420]:
dictClean2 = {}
listaCustomers2 = data['CustomerID'].unique()
listaCustomers2 = sorted(listaCustomers2)
for x in listaCustomers2[:7]:
    similares = tableDistances2[x].sort_values(ascending=False)[1:7]
    lst_cust = list(similares.index)
    data_step5 = customer.loc[lst_cust]
    step6 = data_step5.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
    tx = table[x]
    tx = tx[(tx==0)]
    step7 = pd.merge(tx, step6, left_index=True, right_index=True).sort_values(by="Quantity",ascending=False)
    re={x:step7.index[0:5]}
    dictClean2.update(re)
    
dictClean2

{33: Index(['Kiwi', 'Chips Potato Salt Vinegar 43g', 'Knife Plastic - White',
        'Pasta - Orecchiette', 'Muffin - Zero Transfat'],
       dtype='object', name='ProductName'),
 200: Index(['Langers - Ruby Red Grapfruit', 'Muffin Batt - Blueberry Passion',
        'Otomegusa Dashi Konbu', 'Pasta - Orecchiette',
        'Pasta - Cheese / Spinach Bauletti'],
       dtype='object', name='ProductName'),
 264: Index(['V8 - Berry Blend', 'Water - Mineral, Natural',
        'French Pastry - Mini Chocolate', 'Wine - Toasted Head',
        'Squid - Tubes / Tenticles 10/20'],
       dtype='object', name='ProductName'),
 356: Index(['Bread - English Muffin', 'Olive - Spread Tapenade', 'Bagel - Plain',
        'Pork - Hock And Feet Attached', 'Oil - Shortening - All - Purpose'],
       dtype='object', name='ProductName'),
 412: Index(['Cake - Mini Cheesecake', 'Butter - Unsalted',
        'Salmon - Atlantic, Skin On', 'Salsify, Organic', 'Durian Fruit'],
       dtype='object', name='ProductName

In [421]:
newDataFrame2 = pd.DataFrame(dictClean2)
newDataFrame2

,33,200,264,356,412,464,477
0,Kiwi,Langers - Ruby Red Grapfruit,V8 - Berry Blend,Bread - English Muffin,Cake - Mini Cheesecake,Bread - Bistro White,Olive - Spread Tapenade
1,Chips Potato Salt Vinegar 43g,Muffin Batt - Blueberry Passion,"Water - Mineral, Natural",Olive - Spread Tapenade,Butter - Unsalted,Broom - Corn,Apricots Fresh
2,Knife Plastic - White,Otomegusa Dashi Konbu,French Pastry - Mini Chocolate,Bagel - Plain,"Salmon - Atlantic, Skin On",Knife Plastic - White,Squid U5 - Thailand
3,Pasta - Orecchiette,Pasta - Orecchiette,Wine - Toasted Head,Pork - Hock And Feet Attached,"Salsify, Organic",Wonton Wrappers,"Pasta - Penne, Rigate, Dry"
4,Muffin - Zero Transfat,Pasta - Cheese / Spinach Bauletti,Squid - Tubes / Tenticles 10/20,Oil - Shortening - All - Purpose,Durian Fruit,Flour - Whole Wheat,Sauce - Hollandaise


In [422]:
newDataFrame = pd.DataFrame(dictClean)
newDataFrame

,33,200,264,356,412,464,477
0,Soup - Campbells Bean Medley,Soup - Campbells Bean Medley,"Veal - Inside, Choice",Butter - Unsalted,Wine - Blue Nun Qualitatswein,Butter - Unsalted,"Pepper - Black, Whole"
1,Wine - Blue Nun Qualitatswein,Muffin - Carrot Individual Wrap,Soupfoamcont12oz 112con,"Veal - Inside, Choice",Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Sprouts - Baby Pea Tendrils
2,Knife Plastic - White,Dried Figs,"Pepper - Black, Whole",Sardines,Olive - Spread Tapenade,Knife Plastic - White,Wine - Toasted Head
3,Beef Wellington,Pork - Kidney,Lamb - Ground,"Nut - Chestnuts, Whole",Sprouts - Baby Pea Tendrils,"Sauce - Gravy, Au Jus, Mix","Cheese - Brie, Triple Creme"
4,Wine - Ej Gallo Sierra Valley,Wanton Wrap,Bread - Italian Roll With Herbs,Lamb - Ground,Wine - Chardonnay South,Yoghurt Tubes,"Oranges - Navel, 72"


In [423]:
# Con CORRELATION me dan productos distintos, pero antes había usado CITYBLOCK y me daban productos iguales
# a los recomendados usando EUCLIDEAN.